# Relational Data 

In this task, an agent is given access to a set of tools that can be used to make queries across 3 relational tables.

The tables contain information about users, locations and foods. The agent must answer questions about the data using the provided tools.

The underlying data looks like this (showing first 2 records)

User data:

|  id  |   name   |       email       | location | favorite_color | favorite_foods |
| ---- | -------- | ----------------- | -------- | --------------- | --------------- |
|  1   |  Alice   | alice@gmail.com  |    1     |      red        |   [1, 2, 3]    |
|  21  |   Bob    | bob@hotmail.com  |    2     |     orange      |   [4, 5, 6]    |


Location Data:

|  id  |    city      |     current_time      |            current_weather               |
| ---- | ------------ | --------------------- | --------------------------------------- |
|  1   |  New York    | 2023-11-14 10:30 AM  | Partly Cloudy, Temperature: 68°F       |
|  2   | Los Angeles  | 2023-11-14 7:45 AM   | Sunny, Temperature: 75°F               |



Food data:

|  id  |    name    | calories | allergic_ingredients     |
| ---- | ---------- | -------- | ------------------------ |
|  1   |   Pizza    |   285    |   ["Gluten", "Dairy"]    |
|  2   | Chocolate  |    50    |   ["Milk", "Soy"]        |


The tools allow to look up information based on ids (e.g., `get_user_email` takes a user id and returns the email),
and to search (e.g., `find_foods_by_name` takes a food name and returns a list of results).

----------

In [1]:
from langchain_benchmarks import clone_public_dataset, registry

For this code to work, please configure LangSmith environment variables with your credentials.

In [2]:
task = registry["Tool Usage - Relational Data"]

Clone the dataset associaetd with this task

In [3]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Tool Usage - Relational Data already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/df6be6c9-05b3-445e-8836-ebb4aba63826.


## The Environment

Let's check the environment

In [4]:
env = task.create_environment()
env.tools[:5]

[StructuredTool(name='get_user_name', description="get_user_name(user_id: int) -> str - Get the name of the user with the given user ID.\n\n        Args:\n            user_id: The user's ID.\n\n        Returns:\n            The user's name.", args_schema=<class 'pydantic.v1.main.get_user_nameSchemaSchema'>, handle_tool_error=True, func=<function get_available_functions.<locals>.get_user_name at 0x7fbb0e864f40>),
 StructuredTool(name='list_user_ids', description='list_user_ids() -> List[str] - List all the user IDs.', args_schema=<class 'pydantic.v1.main.list_user_idsSchemaSchema'>, handle_tool_error=True, func=<function get_available_functions.<locals>.list_user_ids at 0x7fbb0e864fe0>),
 StructuredTool(name='find_users_by_name', description='find_users_by_name(name: str) -> List[langchain_benchmarks.tool_usage.tasks.relational_data.SearchHit] - Find users with the given name.\n\n        Args:\n            name: The name to search for.\n\n        Returns:\n            The list of matchi

In [5]:
env.tools[0].invoke({"user_id": 21})

'Bob'

In [6]:
env.tools[3].invoke({"city": "LA"})

[{'id': 2, 'city': 'Los Angeles'},
 {'id': 1, 'city': 'New York'},
 {'id': 3, 'city': 'Chicago'},
 {'id': 4, 'city': 'Houston'},
 {'id': 5, 'city': 'Miami'}]

## Agent Factory

For evaluation, we need an agent factory that will create a new instance of an agent executor for every evaluation run.

The `AgentExecutor` should accept `question` as an input and include the fields `output`, `intermediate_steps` and potentially `state` in its response -- for this we
will wrap the agent executor in an adapter (`apply_agent_executor_adapter`) that will help match the expected schema.

Please reference the LangChain documentation to see how to [use and implement agents](https://python.langchain.com/docs/modules/agents/)

In [7]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI

from langchain_benchmarks.schema import ExtractionTask
from langchain_benchmarks.tool_usage.agents import apply_agent_executor_adapter

In [8]:
class AgentFactory:
    def __init__(self, task: ExtractionTask, model: str) -> None:
        self.task = task
        self.model = model

    def __call__(self):
        # This factory creates a new environment for every agent run.
        # The reason is that the environment may be associated with an environment state (e.g., typewriter)
        # which is changed by the actions of the agent.
        # At the end of the run, the environment state will be read.
        env = task.create_environment()  # Create a new environment for every agent run!
        tools = env.tools
        llm = ChatOpenAI(temperature=0, model=self.model)
        agent_executor = initialize_agent(
            tools,
            llm,
            agent=AgentType.OPENAI_FUNCTIONS,
            return_intermediate_steps=True,
            handle_parsing_errors=True,
        )
        # Apply the adapters so that inputs and outputs match dataset schema
        # state_reader automatically adds the state of the environment at the end of the run.
        return apply_agent_executor_adapter(agent_executor, state_reader=env.read_state)

In [9]:
models = ["gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-4-32k-0613"]
agent_factory = AgentFactory(task, models[0])

Let's test that our agent works

In [10]:
from langchain import globals

globals.set_verbose(True)

In [11]:
agent = agent_factory()
agent.invoke({"question": "whats the weather in LA?"})



> Entering new AgentExecutor chain...

Invoking: `find_locations_by_name` with `{'city': 'Los Angeles'}`


[{'id': 2, 'city': 'Los Angeles'}, {'id': 4, 'city': 'Houston'}, {'id': 1, 'city': 'New York'}, {'id': 3, 'city': 'Chicago'}, {'id': 5, 'city': 'Miami'}]
Invoking: `get_weather_at_location` with `{'location_id': 2}`


Sunny, Temperature: 75°FThe weather in Los Angeles is sunny with a temperature of 75°F.

> Finished chain.


{'input': 'whats the weather in LA?',
 'output': 'The weather in Los Angeles is sunny with a temperature of 75°F.',
 'intermediate_steps': [(AgentActionMessageLog(tool='find_locations_by_name', tool_input={'city': 'Los Angeles'}, log="\nInvoking: `find_locations_by_name` with `{'city': 'Los Angeles'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"city":"Los Angeles"}', 'name': 'find_locations_by_name'}})]),
   [{'id': 2, 'city': 'Los Angeles'},
    {'id': 4, 'city': 'Houston'},
    {'id': 1, 'city': 'New York'},
    {'id': 3, 'city': 'Chicago'},
    {'id': 5, 'city': 'Miami'}]),
  (AgentActionMessageLog(tool='get_weather_at_location', tool_input={'location_id': 2}, log="\nInvoking: `get_weather_at_location` with `{'location_id': 2}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"location_id":2}', 'name': 'get_weather_at_location'}})]),
   'Sunny, Temperature: 75°F')]}

In [12]:
globals.set_verbose(False)

## Eval

Let's evaluate an agent now

In [13]:
import uuid

from langsmith.client import Client

from langchain_benchmarks.tool_usage import get_eval_config

experiment_uuid = uuid.uuid4().hex[:4]

client = Client()

models = ["gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-4-0613"]

for model in models:
    print()
    eval_config = get_eval_config()
    agent_factory = AgentFactory(task, model=model)
    test_run = client.run_on_dataset(
        dataset_name=task.name,
        llm_or_chain_factory=agent_factory,
        evaluation=eval_config,
        verbose=False,
        project_name=f"tool-usage-relational-data-{model}-{experiment_uuid}",
        tags=[model],
        project_metadata={
            "model": model,
            "arch": "openai-functions-agent",
            "id": experiment_uuid,
        },
    )


View the evaluation results for project 'tool-usage-relational-data-gpt-3.5-turbo-1106-8258' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/8aae8e36-720a-42c8-8540-5d5475e7181e?eval=true

View all tests for Dataset Tool Usage - Relational Data at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/df6be6c9-05b3-445e-8836-ebb4aba63826
[------------------------------------------------->] 21/21
View the evaluation results for project 'tool-usage-relational-data-gpt-3.5-turbo-0613-8258' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/d8773df1-b054-41e4-a947-7b256ca8738b?eval=true

View all tests for Dataset Tool Usage - Relational Data at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/df6be6c9-05b3-445e-8836-ebb4aba63826
[------------------------------------------------->] 21/21
View the evaluation results for project 'tool-usage-relational-data-gpt-4-0613-8258' at:


## Inspect

Here, we'll take a look at the underlying results a little bit.

In [29]:
import pandas as pd
from langsmith.client import Client

client = Client()
projects = list(client.list_projects(reference_dataset_name="Multiverse Math"))

dfs = []
for project in projects:
    first_root_run = next(
        client.list_runs(project_name=project.name, execution_order=1)
    )
    # Temporary way to get tag information
    tags = first_root_run.tags
    test_results = client.get_test_results(project_name=project.name)
    test_results["model"] = tags[0]
    dfs.append(test_results)


df = pd.concat(dfs)

In [30]:
count_df = df.groupby("model").size().to_frame("n")
df.groupby("model")[
    [
        "feedback.correctness",
        "feedback.Intermediate steps correctness",
        "execution_time",
        "feedback.# steps / # expected steps",
    ]
].mean().join(count_df)

,feedback.correctness,feedback.Intermediate steps correctness,execution_time,feedback.# steps / # expected steps,n
model,,,,,
gpt-3.5-turbo-0613,0.3,0.8,8.308014,1.03333,10
gpt-3.5-turbo-1106,0.1,0.7,54.992810,0.93332,10
gpt-4-0613,0.0,0.6,8.554704,0.79999,10


In [24]:
import pandas as pd
from langsmith.client import Client

client = Client()
projects = list(
    client.list_projects(reference_dataset_name="Tool Usage - Relational Data")
)

dfs = []
for project in projects:
    first_root_run = next(
        client.list_runs(project_name=project.name, execution_order=1)
    )
    # Temporary way to get tag information
    tags = first_root_run.tags
    test_results = client.get_test_results(project_name=project.name)
    test_results["model"] = tags[0]
    dfs.append(test_results)


df = pd.concat(dfs)

In terms of function usage, gpt-4 uses more calls than is strictly necessary (`feedback.# steps / # expected steps` is > 1). However, it's doing a pretty good job.

The gpt-3.5 models do not use tools enough (`feedback.# steps / # expected steps` is < 1) and as a result do a worse job at the task.

Note: The intermediate step correctness happens to have the same average for the 3 models -- this is just a coincidence you can confirm by inspecting underlying results.

In [25]:
count_df = df.groupby("model").size().to_frame("n")
df.groupby("model")[
    [
        "feedback.correctness",
        "feedback.Intermediate steps correctness",
        "execution_time",
        "feedback.# steps / # expected steps",
    ]
].mean().join(count_df)

,feedback.correctness,feedback.Intermediate steps correctness,execution_time,feedback.# steps / # expected steps,n
model,,,,,
gpt-3.5-turbo-0613,0.714286,0.714286,4.829506,0.825390,21
gpt-3.5-turbo-1106,0.857143,0.714286,5.464218,0.965871,21
gpt-4-0613,0.952381,0.714286,8.544358,1.037300,21
